
# Vehicle Routing with Time Windows (VRPTW) — MIP + QUBO on One Dataset (Colab-Ready)

This notebook tackles **VRPTW** on a single synthetic dataset using two complementary approaches:

1. **Classical MIP (PuLP)**  
   - Binary arc decisions \(x_{i,j,k}\) for vehicle \(k\) traveling from node \(i\) to \(j\).  
   - Arrival times \(t_{i,k}\), and capacity/order variables.  
   - Constraints: **each customer served once**, **flow conservation**, **vehicle capacity**, and **time windows**.  
   - Objective: **minimize total travel time**.

2. **QUBO (assignment-first, sequencing-second)**  
   - Binary \(z_{i,k}\): assign customer \(i\) to vehicle \(k\).  
   - Penalties for **single assignment**, **capacity**, and **soft duration/time-window limits**.  
   - Solve with **simulated annealing** (`neal`), then **sequence each vehicle** with a greedy time-window heuristic.

We then compare total route durations and visualize the routes for both methods.


In [ ]:

# Install dependencies if needed (works in Colab)
def _silent_imports():
    flags = {"pulp": False, "dimod": False, "neal": False}
    try:
        import pulp
        flags["pulp"] = True
    except Exception:
        pass
    try:
        import dimod
        flags["dimod"] = True
    except Exception:
        pass
    try:
        import neal
        flags["neal"] = True
    except Exception:
        pass
    return flags

flags = _silent_imports()
if not flags["pulp"]:
    %pip -q install pulp
if not flags["dimod"] or not flags["neal"]:
    %pip -q install dimod neal

flags = _silent_imports()
print("PuLP:", flags["pulp"], "| dimod:", flags["dimod"], "| neal:", flags["neal"])


In [ ]:

# ==== One Synthetic VRPTW Dataset ====
import numpy as np, pandas as pd

rng = np.random.default_rng(77)

N = 12             # customers
K = 3              # vehicles
Q = 30             # vehicle capacity
speed = 1.0        # distance -> travel time
service_time = 2.0 # service time at each customer
depot = 0

# Coordinates (random clusters)
centers = np.array([[0,0],[25,0],[12,18]], dtype=float)
customers_xy = []
labels = rng.integers(0, len(centers), size=N)
for i in range(N):
    cx, cy = centers[labels[i]]
    customers_xy.append([cx, cy] + rng.normal(0, 2.5, size=2))
customers_xy = np.array(customers_xy)

# Build nodes: 0 = depot, 1..N = customers
nodes_xy = np.vstack([np.array([[0.0, 0.0]]), customers_xy])

# Demands and time windows (early, late)
demand = rng.integers(2, 8, size=N)  # 2..7 units
# Time windows center around distance from depot + slack
dist_depot = np.linalg.norm(nodes_xy[1:] - nodes_xy[0], axis=1)
earliest = (dist_depot * 0.8).round(1) + rng.uniform(0, 3, size=N)
latest = earliest + rng.uniform(8, 15, size=N)
tw = np.vstack([earliest, latest]).T

# Distance/time matrix
coords = nodes_xy
Dmat = np.linalg.norm(coords[:,None,:] - coords[None,:,:], axis=2)  # Euclidean
Tmat = Dmat / speed

df_nodes = pd.DataFrame({
    "node": list(range(N+1)),
    "x": coords[:,0],
    "y": coords[:,1],
    "demand": [0] + demand.tolist(),
    "earliest": [0.0] + earliest.round(1).tolist(),
    "latest": [1e6] + latest.round(1).tolist()  # depot wide window
})

print("Customers:", N, "| Vehicles:", K, "| Capacity:", Q)
df_nodes.head()



## Part 1 — Classical MIP (PuLP) with Time Windows

**Variables:**
- \(x_{i,j,k}\in\{0,1\}\): vehicle \(k\) travels directly from \(i\) to \(j\)  
- \(t_{i,k}\ge 0\): arrival time of vehicle \(k\) at node \(i\)  
- \(w_{i,k}\): load carried just after serving node \(i\) (for capacity tracking)  
- \(u_{i,k}\): order/index (MTZ-style) to eliminate subtours per vehicle

**Constraints:** demand served once, flow conservation, capacity, time propagation with service times, and time windows.


In [ ]:

import numpy as np, pandas as pd

try:
    import pulp
    HAVE_PULP = True
except Exception:
    HAVE_PULP = False

def solve_vrptw_mip(N, K, Q, Tmat, demand, tw, service_time=2.0, depot=0):
    # Nodes 0..N; customers 1..N
    nodes = list(range(N+1))
    customers = list(range(1, N+1))

    prob = pulp.LpProblem("VRPTW", pulp.LpMinimize)

    # Decision vars
    x = pulp.LpVariable.dicts("x", (nodes, nodes, range(K)), lowBound=0, upBound=1, cat="Binary")
    t = pulp.LpVariable.dicts("t", (nodes, range(K)), lowBound=0, cat="Continuous")
    w = pulp.LpVariable.dicts("w", (nodes, range(K)), lowBound=0, upBound=Q, cat="Continuous")
    u = pulp.LpVariable.dicts("u", (customers, range(K)), lowBound=0, upBound=N, cat="Continuous")

    # Objective: minimize total travel time
    prob += pulp.lpSum(Tmat[i][j] * x[i][j][k] for i in nodes for j in nodes if i!=j for k in range(K))

    BIGM = 1e4

    # Each customer serviced exactly once
    for j in customers:
        prob += pulp.lpSum(x[i][j][k] for i in nodes if i!=j for k in range(K)) == 1

    # Flow conservation for each vehicle
    for k in range(K):
        # depart depot once and return once
        prob += pulp.lpSum(x[depot][j][k] for j in customers) == 1
        prob += pulp.lpSum(x[i][depot][k] for i in customers) == 1
        for h in customers:
            prob += pulp.lpSum(x[i][h][k] for i in nodes if i!=h) == pulp.lpSum(x[h][j][k] for j in nodes if j!=h)

    # Capacity propagation
    for k in range(K):
        w[depot][k].lowBound = 0
        for i in nodes:
            for j in customers:
                if i == j: continue
                # if travel i->j then w_j = w_i + demand_j
                prob += w[j][k] >= w[i][k] + demand[j-1] - BIGM*(1 - x[i][j][k])
        for i in customers:
            prob += w[i][k] >= demand[i-1]
            prob += w[i][k] <= Q
        prob += w[depot][k] <= Q

    # Time windows and time propagation
    for k in range(K):
        t[depot][k].lowBound = 0
        for i in customers:
            ei, li = tw[i-1]
            prob += t[i][k] >= float(ei)
            prob += t[i][k] <= float(li)
        for i in nodes:
            for j in customers + [depot]:
                if i == j: continue
                travel = Tmat[i][j]
                service = (service_time if i != depot else 0.0)
                prob += t[j][k] >= t[i][k] + service + travel - BIGM*(1 - x[i][j][k])

    # Subtour elimination (MTZ-style) per vehicle
    for k in range(K):
        for i in customers:
            for j in customers:
                if i == j: continue
                prob += u[i][k] - u[j][k] + (N)*x[i][j][k] <= N-1

    # Forbid self-loops
    for k in range(K):
        for i in nodes:
            prob += x[i][i][k] == 0

    # Solve
    _ = prob.solve(pulp.PULP_CBC_CMD(msg=False))
    status = pulp.LpStatus[prob.status]
    if status not in ("Optimal", "Feasible"):
        return status, None, None

    x_sol = {(i,j,k): int(round(pulp.value(x[i][j][k]) or 0)) for i in nodes for j in nodes for k in range(K)}
    t_sol = {(i,k): float(pulp.value(t[i][k]) or 0.0) for i in nodes for k in range(K)}
    return status, x_sol, t_sol

def heuristic_vrptw(N, K, Q, Tmat, demand, tw, service_time=2.0, depot=0):
    # Greedy insertion by earliest latest time; simple feasibility checks
    nodes = list(range(N+1))
    customers = set(range(1, N+1))
    routes = [[] for _ in range(K)]
    loads = [0]*K
    times = [0.0]*K

    while customers:
        j = min(customers, key=lambda c: tw[c-1][1])  # tightest latest window
        placed = False
        for k in range(K):
            if loads[k] + demand[j-1] > Q:
                continue
            # try to append at end
            last = depot if len(routes[k])==0 else routes[k][-1]
            arrival = times[k] + (service_time if last!=depot else 0.0) + Tmat[last][j]
            arrival = max(arrival, tw[j-1][0])
            if arrival <= tw[j-1][1]:
                routes[k].append(j)
                loads[k] += demand[j-1]
                times[k] = arrival
                placed = True
                break
        if not placed:
            # place into the least loaded vehicle ignoring time (soft)
            k = int(np.argmin(loads))
            routes[k].append(j)
            loads[k] += demand[j-1]
            times[k] += Tmat[routes[k][-2] if len(routes[k])>1 else depot][j] + service_time
        customers.remove(j)

    # Build x_sol
    x_sol = {}
    for k in range(K):
        seq = [depot] + routes[k] + [depot]
        for a,b in zip(seq[:-1], seq[1:]):
            x_sol[(a,b,k)] = 1
    return "Heuristic", x_sol, None

if HAVE_PULP:
    mip_status, x_mip, t_mip = solve_vrptw_mip(N, K, Q, Tmat, demand, tw, service_time, depot)
else:
    mip_status, x_mip, t_mip = heuristic_vrptw(N, K, Q, Tmat, demand, tw, service_time, depot)

print("MIP/Heuristic status:", mip_status)


In [ ]:

# Extract MIP/Heuristic routes and total duration
import pandas as pd, numpy as np

def extract_routes(x_sol, N, K, depot=0):
    routes = []
    for k in range(K):
        # Build adjacency
        succ = {i:j for (i,j,kk),v in x_sol.items() if kk==k and v==1}
        route = []
        cur = depot
        visited = set([depot])
        while True:
            if cur not in succ:
                break
            nxt = succ[cur]
            if nxt==depot:
                break
            route.append(nxt)
            if nxt in visited:
                break
            visited.add(nxt)
            cur = nxt
        routes.append(route)
    return routes

def route_duration(route, Tmat, service_time, depot=0):
    if not route: return 0.0
    seq = [depot] + route + [depot]
    total = 0.0
    for a,b in zip(seq[:-1], seq[1:]):
        total += Tmat[a][b]
        if a != depot:
            total += service_time
    return total

routes_mip = extract_routes(x_mip, N, K, depot)
dur_mip = sum(route_duration(r, Tmat, service_time, depot) for r in routes_mip)

print("Routes (MIP/Heuristic):", routes_mip)
print("Total duration (MIP/Heuristic):", round(dur_mip,2))



## Part 2 — QUBO (Assignment to Vehicles) + Greedy Sequencing

We construct a compact **QUBO** to **assign** each customer \(i\) to a vehicle \(k\):
- **Single assignment:** \(\sum_k z_{i,k} = 1\)
- **Capacity:** \(\sum_i d_i z_{i,k} \le Q\) (soft via squared penalty)
- **Soft duration/time-window:** penalize if approximate route duration for vehicle \(k\) exceeds a limit \(T_{\max}\)

**Objective (compact surrogate):**
- Encourage clusters of nearby customers on the same vehicle: \(\sum_k \sum_{i<j} D_{ij}\, z_{i,k} z_{j,k}\)
- Add linear bias to prefer nearer customers overall: \(\sum_{i,k} \beta\, D_{0i}\, z_{i,k}\)

After solving, we **sequence** each vehicle's customers by a greedy time-window heuristic.


In [ ]:

from collections import defaultdict
import numpy as np, pandas as pd
import dimod, neal

customers = list(range(1, N+1))

# Penalties
A = 10.0  # single assignment
Ccap = 0.5  # capacity
Cdur = 0.5  # duration
beta = 0.1  # linear bias
Tmax = float(np.percentile(Tmat[0,1:], 75) * (N/K) * 2 + service_time*(N/K))  # rough per-vehicle time limit

def zkey(i,k): return (i-1)*K + k  # unique id for QUBO variable

Q = defaultdict(float)

# (1) Single assignment per customer
for i in customers:
    # (1 - sum_k z_{ik})^2 = 1 - 2 sum z + sum z + 2 sum_{k<l} z_k z_l
    for k in range(K):
        vid = zkey(i,k)
        Q[(vid, vid)] += -A
    for k in range(K):
        for l in range(k+1, K):
            Q[(zkey(i,k), zkey(i,l))] += 2*A

# (2) Capacity penalty per vehicle: (sum_i d_i z_{ik} - Q)^2
for k in range(K):
    for i in customers:
        Q[(zkey(i,k), zkey(i,k))] += Ccap * (demand[i-1]**2 - 2*Q*demand[i-1])
    for i in customers:
        for j in range(i+1, N+1):
            Q[(zkey(i,k), zkey(j,k))] += Ccap * (2 * demand[i-1] * demand[j-1])

# (3) Duration surrogate per vehicle using pairwise distances + depot bias
for k in range(K):
    for i in customers:
        Q[(zkey(i,k), zkey(i,k))] += Cdur * (beta * Dmat[0][i])  # linear bias
    for i in customers:
        for j in range(i+1, N+1):
            Q[(zkey(i,k), zkey(j,k))] += Cdur * Dmat[i][j]

# (4) Soft limit on number of assigned customers per vehicle (optional)
target_per_vehicle = N / K
Spen = 0.2
for k in range(K):
    for i in customers:
        Q[(zkey(i,k), zkey(i,k))] += Spen * (1 - 2*target_per_vehicle)
    for i in customers:
        for j in range(i+1, N+1):
            Q[(zkey(i,k), zkey(j,k))] += 2 * Spen

# Solve QUBO
bqm = dimod.BinaryQuadraticModel.from_qubo(dict(Q))
sampleset = neal.SimulatedAnnealingSampler().sample(bqm, num_reads=3000)
best = sampleset.first

z = np.array([best.sample.get(zkey(i,k), 0) for i in customers for k in range(K)]).reshape(N, K)
assign_qubo = {i: int(np.argmax(z[i-1])) for i in customers}
sizes = [sum(1 for i in customers if assign_qubo[i]==k) for k in range(K)]

print("QUBO vehicle sizes:", sizes)


In [ ]:

# Greedy time-window sequencing per vehicle cluster
import numpy as np

def sequence_cluster(cluster, Tmat, tw, service_time, depot=0):
    route = []
    cur = depot
    time = 0.0
    remaining = set(cluster)
    while remaining:
        # choose feasible nearest by arrival time
        cand = []
        for j in remaining:
            arr = time + (service_time if cur!=depot else 0.0) + Tmat[cur][j]
            start = max(arr, tw[j-1][0])
            feasible = start <= tw[j-1][1]
            cand.append((feasible, start, Tmat[cur][j], j))
        # prefer feasible with earliest start, else nearest
        cand.sort(key=lambda t: (not t[0], t[1], t[2]))
        _, start, _, j = cand[0]
        route.append(j)
        time = start
        cur = j
        remaining.remove(j)
    return route

routes_qubo = []
for k in range(K):
    cluster = [i for i in range(1, N+1) if assign_qubo[i]==k]
    if cluster:
        r = sequence_cluster(cluster, Tmat, tw, service_time, depot)
    else:
        r = []
    routes_qubo.append(r)

dur_qubo = sum(route_duration(r, Tmat, service_time, depot=0) for r in routes_qubo)

print("Routes (QUBO+greedy):", routes_qubo)
print("Total duration (QUBO+greedy):", round(dur_qubo,2))


In [ ]:

# Plot routes for MIP/Heuristic and QUBO solutions
import matplotlib.pyplot as plt
import numpy as np

def plot_routes(routes, title):
    plt.figure()
    xs = df_nodes["x"].values; ys = df_nodes["y"].values
    plt.scatter(xs[1:], ys[1:])
    plt.scatter([xs[0]],[ys[0]], marker='s')
    for r in routes:
        seq = [0] + r + [0]
        for a,b in zip(seq[:-1], seq[1:]):
            plt.plot([xs[a], xs[b]], [ys[a], ys[b]])
    plt.title(title)
    plt.xlabel("x"); plt.ylabel("y")
    plt.tight_layout()

plot_routes(routes_mip, "VRPTW — MIP/Heuristic Routes")
plot_routes(routes_qubo, "VRPTW — QUBO Assignment + Greedy Routes")



## Wrap-up

- **MIP** solves VRPTW with exact time-window and capacity constraints (on small instances).  
- **QUBO** assigns customers to vehicles with capacity and compactness penalties, then **sequences** with a simple greedy heuristic.  
- The QUBO surrogate uses pairwise distance and soft penalties, trading optimality for a quantum-friendly form that scales more smoothly.

Try adjusting `N`, `K`, and `Q` at the top, and re-run to see how both methods behave.
